In [ ]:

import os
import time

from fpdf import FPDF
import pandas as pd
import plotly.express as px
import math
from numpy import nan

import numpy as np
import requests
import re
import json
import datetime

from yahoofinancials import YahooFinancials

In [ ]:
#@title DATA old

def get_information(ticker):
    site = "https://finance.yahoo.com/quote/" + ticker + "/profile?p=" + ticker
    html = requests.get(url= site, headers = {'User-agent': 'Mozilla/5.0'}).text
    json_str = html.split('root.App.main =')[1].split('(this)')[0].split(';\n}')[0].strip()
    data = json.loads(json_str)

    company_info ={}
    company_info['ticker'] = ticker
    try:company_info['name'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['quoteType']['longName']
    except:company_info['name'] = ticker
    try:company_info['currency'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['price']['currency']
    except:company_info['currency'] = 'unknown currency'
    try:company_info['currency_symbol'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['price']['currencySymbol']
    except : company_info['currency_symbol'] = 'unknown currency symbol'
    try: company_info['info'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['longBusinessSummary']
    except: company_info['info'] = None
    try: company_info['sector'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['sector']
    except: company_info['sector'] = None
    try: company_info['subsector'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['industry']
    except: company_info['subsector'] = None
    try: company_info['country'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['country']
    except: company_info['country'] = None
    try: company_info['state'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['state']
    except: company_info['state'] = None
    try: company_info['city'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['city']
    except: company_info['city'] = None
    try: company_info['zip'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['zip']
    except: company_info['zip'] = None
    try: company_info['address'] = data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['address1']
    except: company_info['address'] = None  

    return company_info
    

def get_financial_statement(ticker, yearly = True):
    
    '''Scrapes the cash financial statement from Yahoo Finance for an input ticker 
    
       @param: ticker
    '''
    
    financials_site = "https://finance.yahoo.com/quote/" + ticker + "/financials?p=" + ticker

    headers = {'User-agent': 'Mozilla/5.0'}

    html = requests.get(url = financials_site, headers = headers).text

    json_str = html.split('root.App.main =')[1].split('(this)')[0].split(';\n}')[0].strip()
    
    try:
        data = json.loads(json_str)[
            'context']['dispatcher']['stores']['QuoteSummaryStore']
    except:
        json_info = '{}'
    else:
        # return data
        new_data = json.dumps(data).replace('{}', 'null')
        new_data = re.sub(r'\{[\'|\"]raw[\'|\"]:(.*?),(.*?)\}', r'\1', new_data)

        json_info = json.loads(new_data)
    
    if yearly:
        temp_is = json_info["incomeStatementHistory"]["incomeStatementHistory"]
        temp_bs = json_info["balanceSheetHistory"]["balanceSheetStatements"]
        temp_cf = json_info["cashflowStatementHistory"]["cashflowStatements"]
    else:
        temp_is = json_info["incomeStatementHistoryQuarterly"]["incomeStatementHistory"]
        temp_bs = json_info["balanceSheetHistoryQuarterly"]["balanceSheetStatements"]
        temp_cf = json_info["cashflowStatementHistoryQuarterly"]["cashflowStatements"]
        
    
    df_is = pd.DataFrame(temp_is)
    df_bs = pd.DataFrame(temp_bs)
    df_cf = pd.DataFrame(temp_cf)
    
    if (not df_is.empty):

        if 'maxAge' in df_is.keys(): del df_is["maxAge"]

        df_is.set_index("endDate", inplace=True)
        df_is.index = pd.to_datetime(df_is.index, unit="s")

        df_is = df_is.transpose()
        df_is.index.name = "Breakdown"

    if (not df_bs.empty):

        if 'maxAge' in df_bs.keys(): del df_bs["maxAge"]

        df_bs.set_index("endDate", inplace=True)
        df_bs.index = pd.to_datetime(df_bs.index, unit="s")

        df_bs = df_bs.transpose()
        df_bs.index.name = "Breakdown"

    if (not df_cf.empty):

        if 'maxAge' in df_cf.keys(): del df_cf["maxAge"]

        df_cf.set_index("endDate", inplace=True)
        df_cf.index = pd.to_datetime(df_cf.index, unit="s")

        df_cf = df_cf.transpose()
        df_cf.index.name = "Breakdown"

    fs = pd.concat([df_is, df_bs, df_cf], axis=0)
    
    fs['name']=fs.index
    fs.drop_duplicates(inplace=True)
    fs.drop(columns=['name'], inplace=True)

    fs = fs.T
    
    #fs['date'] = pd.to_datetime(fs.index).strftime("%d %B, %Y")

    fs['date'] = pd.to_datetime(fs.index).strftime("%Y-%m-%d")
    #'2018-12-31'
    
    fs = fs.to_dict('list')               

    return fs

def get_add_info_annually(ticker):
    balance_sheet_site = "https://finance.yahoo.com/quote/" + ticker + "/balance-sheet?p=" + ticker
    html = requests.get(url=balance_sheet_site, headers = {'User-agent': 'Mozilla/5.0'}).text
    json_str = html.split('root.App.main =')[1].split('(this)')[0].split(';\n}')[0].strip()
    data_bs = json.loads(json_str)

    date_index = []
    values = []
    for i in data_bs['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries']['annualTotalDebt']:
        if i != None:
            date_index.append(i['asOfDate'])
            values.append(i['reportedValue']['raw'])

    add_df = pd.DataFrame({'TotalDebt':values})

    add_df['date'] = date_index

    date_index = []
    values = []
    for i in data_bs['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries']['annualWorkingCapital']:
        if i != None:
            date_index.append(i['asOfDate'])
            values.append(i['reportedValue']['raw']) 

    add_df1 = pd.DataFrame({'WorkingCapital':values})

    add_df1['date'] = date_index

    add_df = add_df.merge(add_df1, how='outer', left_on='date', right_on='date')

    date_index = []
    values = []
    for i in data_bs['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries']['annualAccountsReceivable']:
        if i != None:
            date_index.append(i['asOfDate'])
            values.append(i['reportedValue']['raw'])

    add_df1 = pd.DataFrame({'AccountsReceivable':values})

    add_df1['date'] = date_index

    add_df = add_df.merge(add_df1, how='outer', left_on='date', right_on='date')

    cash_flow_site = "https://finance.yahoo.com/quote/" + ticker + "/cash-flow?p=" + ticker
    html = requests.get(url=cash_flow_site, headers = {'User-agent': 'Mozilla/5.0'}).text
    json_str = html.split('root.App.main =')[1].split('(this)')[0].split(';\n}')[0].strip()
    data_cf = json.loads(json_str)

    date_index = []
    values = []
    for i in data_cf['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries']['annualFreeCashFlow']:
        if i != None:
            date_index.append(i['asOfDate'])
            values.append(i['reportedValue']['raw'])

    add_df1 = pd.DataFrame({'FreeCashFlow':values})

    add_df1['date'] = date_index  

    add_df = add_df.merge(add_df1, how='outer', left_on='date', right_on='date')

    date_index = []
    values = []
    for i in data_cf['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries']['annualFinancingCashFlow']:
        if i != None:
            date_index.append(i['asOfDate'])
            values.append(i['reportedValue']['raw'])

    add_df1 = pd.DataFrame({'FinancingCashFlow':values})

    add_df1['date'] = date_index 

    add_df = add_df.merge(add_df1, how='outer', left_on='date', right_on='date')

    date_index = []
    values = []
    for i in data_cf['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries']['annualOperatingCashFlow']:
        if i != None:
            date_index.append(i['asOfDate'])
            values.append(i['reportedValue']['raw'])

    add_df1 = pd.DataFrame({'OperatingCashFlow':values})

    add_df1['date'] = date_index 

    add_df = add_df.merge(add_df1, how='outer', left_on='date', right_on='date')

    date_index = []
    values = []
    for i in data_cf['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries']['annualSaleOfPPE']:
        if i != None:
            date_index.append(i['asOfDate'])
            values.append(i['reportedValue']['raw'])

    add_df1 = pd.DataFrame({'SaleOfPPE':values})

    add_df1['date'] = date_index 

    add_df = add_df.merge(add_df1, how='outer', left_on='date', right_on='date')  

    add_df['ticker'] = ticker 

    return add_df

def fs_to_df(tck):
    dct=get_financial_statement(tck)
    rtr=pd.DataFrame.from_dict(dct)
    return rtr

def info_to_df(tck):
    dct=get_information(tck)
    rtr1=pd.DataFrame(dct,index=[0])
    rtr2=pd.DataFrame(dct,index=[1])
    rtr3=pd.DataFrame(dct,index=[2])
    rtr4=pd.DataFrame(dct,index=[3])
    return pd.concat([rtr1,rtr2,rtr3,rtr4])


def concat(tck):
    df1=fs_to_df(tck)[::-1].reset_index(drop=True)
    df2=get_add_info_annually(tck)
    df3=info_to_df(tck)
    df=pd.concat([df1,df2,df3], axis=1)
    df = df.loc[:,~df.columns.duplicated()].copy()
    df['date2']=pd.DatetimeIndex(df['date']).year
    return df

# done by Alexandre

def ratios_(tck):
    df=concat(tck)[['ticker', 'date2', 'date']]
    dfc=concat(tck)
    dfc['ebitda'] = dfc['operatingIncome'] + dfc['depreciation'] 
    dfc['EnterpriseValue'] = dfc['totalStockholderEquity'] + dfc['TotalDebt'] - dfc['cash']
    df['ebit_to_interestExpense'] = dfc['ebit'].divide(abs(dfc['interestExpense']))
    df['ebitda_to_interestExpense'] = dfc['ebitda'].divide(abs(dfc['interestExpense']))
    df['FreeOperCF_to_TotalDebt'] = (dfc['ebitda'] - dfc['capitalExpenditures']).divide(dfc['TotalDebt'])
    df['FFO_to_TotalDebt'] = (dfc['ebitda'] + dfc['SaleOfPPE']).divide(dfc['TotalDebt'])
    df['Return_on_Capital'] = (dfc['operatingIncome'] - dfc['incomeTaxExpense']).divide(dfc['EnterpriseValue'])
    df['ebitda_to_Revenue'] = dfc['ebitda'].divide(dfc['totalRevenue'])
    df['TotalDebt_to_EnterpriseValue'] = dfc['TotalDebt'].divide(dfc['EnterpriseValue'])
    df['TotalDebt_to_ebitda'] = dfc['TotalDebt'].divide(dfc['ebitda'])
    df['ROCE'] = dfc['ebit'].divide(dfc['totalAssets'] + dfc['totalCurrentLiabilities'])
    #df['ROIC'] = (dfc['netIncome']-dfc['dividendsPaid']).divide(dfc['TotalDebt'] + dfc['totalStockholderEquity'])
    
    df['DIO'] = (((dfc['inventory'].add(dfc['inventory'].shift())).div(2)).divide(dfc['costOfRevenue'])).mul(365)
    df['DPO'] = (((dfc['accountsPayable'].add(dfc['accountsPayable'].shift())).div(2)).divide(dfc['costOfRevenue'])).mul(365)
    df['DSO'] = (((dfc['AccountsReceivable'].add(dfc['AccountsReceivable'].shift())).div(2)).divide(dfc['totalRevenue'])).mul(365)
    df['InventoryTurnover'] = dfc['costOfRevenue'].div(((dfc['inventory'].add(dfc['inventory'].shift())).div(2)))
    df['CCC'] = df['DIO'] + df['DSO'] - df['DPO']



    return df

In [ ]:
#@title DATA New
ticker = 'AAPL'
aapl = YahooFinancials(ticker)
all_statements_yr =  aapl.get_financial_stmts('annual', ['income', 'cash', 'balance'])
all_statements_qt =  aapl.get_financial_stmts('quarterly', ['income', 'cash', 'balance'])



In [ ]:
def display_formating(df__di):
    
    df_d=df__di.copy()
    
    for name, values in df_d.iteritems():
    
        if name != 'date2':
            if df_d[name].dtype==np.int64 or df_d[name].dtype==np.float64:
                df_d.rename(columns = {name:str(name)+' ($ Millions)'}, inplace = True)

                for index, row in df_d.iterrows():
                    if math.isnan(df_d[str(name)+' ($ Millions)'].loc[index]):
                        pass
                    else:
                        numform=df_d[str(name)+' ($ Millions)'].loc[index]/1000000
                        numform=int(numform)
                        df_d[str(name)+' ($ Millions)'].loc[index]=numform
                    
    return df_d

**REPORT FORMATING**

In [ ]:
#@title REPORT FORMATING

# done
def plot_style(paper_bg='rgba(0,0,0,0)', plot_bg='#e8ecfc', font='#0F1026'):
    plot_style = {'paper_bgcolor':paper_bg, 'plot_bgcolor':plot_bg,
                               'font': {'color':font}}
    return plot_style

# done
def update_tcc_bar3(ticks, Lambda):
    tickers_list=ticks    
    
    df0 = concat(tickers_list[0])
    
    df1 = df0.copy()
    
    df2 = df0.copy()
    
    df0['TCC'] = 'Without TCC'
    
    df1['TCC'] = 'With TCC'
    
    df1['TotalDebt'] = df0['TotalDebt'].mul(1-Lambda)
    df1['interestExpense'] = df0['interestExpense'].mul(1-Lambda)
    df1['operatingIncome'] = df0['operatingIncome'] - abs(df0['interestExpense']).mul(Lambda)
    #df1['ebitda'] = df0['ebitda'] - abs(df0['interestExpense']).mul(Lambda)
    
    df2['TotalDebt'] = df0['TotalDebt'].mul(1-0.8)
    df2['interestExpense'] = df0['interestExpense'].mul(1-0.8)
    df2['operatingIncome'] = df0['operatingIncome'] - abs(df0['interestExpense']).mul(0.8)
    #df2['ebitda'] = df0['ebitda'] - abs(df0['interestExpense']).mul(0.8)
    
    df = pd.concat([df0, df1], axis=0)
    
    df['date'] = pd.to_datetime(df['date'])
    
    df_ratios = df[['ticker', 'date', 'TCC', 'name']]
    df_ratios2 = df[['ticker', 'date']]
    
    #df = df[['ticker', 'date', 'interestExpense', 'TotalDebt', 'ebitda', 'operatingIncome']]
    
    df_ratios['ebit_to_interestExpense'] = df['ebit'].divide(abs(df['interestExpense']))
    df_ratios['ebitda_to_interestExpense'] = (df['operatingIncome'] 
                                              + df['depreciation']).divide(abs(df['interestExpense']))

    
    df_ratios2['ebit_to_interestExpense'] = df2['ebit'].divide(abs(df2['interestExpense']))
    df_ratios2['ebitda_to_interestExpense'] = (df2['operatingIncome'] 
                                              + df2['depreciation']).divide(abs(df2['interestExpense']))

    
    tcc3_max = max(df_ratios2['ebit_to_interestExpense'].max(),
                   df_ratios2['ebitda_to_interestExpense'].max())
    
    ratios_melt = pd.melt(df_ratios, id_vars=['ticker', 'date', 'TCC', 'name'])
                                                          
    ratios_melt['date'] = pd.to_datetime(ratios_melt['date'])
    
    ratios_melt_cols3 = ratios_melt[ratios_melt.variable.isin(['ebit_to_interestExpense', 
                                                'ebitda_to_interestExpense'])]
    
    #ratios_melt_cols3=display_formating(ratios_melt_cols3)
    ratios_melt_cols3['Date_formated']=ratios_melt_cols3['date'].astype(str).str[0:4]

    color_discrete_map=my_palette1
    
    if (Lambda < 0.8):
        tcc3_bar = px.bar(ratios_melt_cols3, x="Date_formated", y="value", facet_row='ticker',
                          facet_col='TCC', hover_name='name',
                          labels={
                              "value":"Ratio",
                              "variable":" ",
                              "Date_formated": ""
                         },
                          color="variable", color_discrete_map= color_discrete_map,
                          barmode="group", range_y=[-0.1*tcc3_max, 1.1*tcc3_max],text_auto='.2s')
    else:    
        #color_discrete_sequence[5] = '#609cd4'
        tcc3_bar = px.bar(ratios_melt_cols3, x="Date_formated", y="value", facet_row='ticker',
                          facet_col='TCC', hover_name='name',
                          labels={
                              "value":"Ratio",
                              "variable":" ",
                              "Date_formated": ""
                         },
                          color='variable', color_discrete_map= color_discrete_map,
                          barmode="group", text_auto='.2s')
    
    tcc3_bar.update_layout(plot_style())
    tcc3_bar.update_layout(
        legend=dict(
            x=0,
            y=1,
            traceorder="reversed",
            bgcolor="#ffffff",
            bordercolor="Black",
            borderwidth=2,
            title=None,
        )
    )
                         
    tcc3_bar.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    tcc3_bar.update_annotations(font_size=25)
    
    return tcc3_bar

# done
def bar2_compare(ticks, Lambda):

    my_palette2 ={'FreeOperCF_to_TotalDebt': '#FFA41B', 'FFO_to_TotalDebt': '#4C71DD'}
    import pandas as pd
    import plotly.express as px
    from plotly.offline import plot

    tickers_list = ticks

    df0 = concat(tickers_list[0])

    df1 = df0.copy()

    df2 = df0.copy()

    df0['TCC'] = 'Without TCC'

    df1['TCC'] = 'With TCC'
    
    df0['ebitda'] = df0['operatingIncome'] + df0['depreciation']    
    #df1['ebitda'] = df1['operatingIncome'] + df1['depreciation']
    #df2['ebitda'] = df2['operatingIncome'] + df2['depreciation']

    df1['TotalDebt'] = df0['TotalDebt'].mul(1 - Lambda)
    df1['interestExpense'] = df0['interestExpense'].mul(1 - Lambda)
    df1['operatingIncome'] = df0['operatingIncome'] - abs(df0['interestExpense']).mul(Lambda)
    df1['ebitda'] = df0['ebitda'] - abs(df0['interestExpense']).mul(Lambda)

    df2['TotalDebt'] = df0['TotalDebt'].mul(1 - 0.8)
    df2['interestExpense'] = df0['interestExpense'].mul(1 - 0.8)
    df2['operatingIncome'] = df0['operatingIncome'] - abs(df0['interestExpense']).mul(0.8)
    df2['ebitda'] = df0['ebitda'] - abs(df0['interestExpense']).mul(0.8)

    df = pd.concat([df0, df1], axis=0)

    df['date'] = pd.to_datetime(df['date'])

    df_ratios = df[['ticker', 'date', 'TCC', 'name']]
    df_ratios2 = df[['ticker', 'date']]

    # df = df[['ticker', 'date', 'interestExpense', 'TotalDebt', 'ebitda', 'operatingIncome']]

    df_ratios['FreeOperCF_to_TotalDebt'] = (df['operatingIncome'] + df['depreciation']
                                            - df['capitalExpenditures']).divide(df['TotalDebt'])
    df_ratios['FFO_to_TotalDebt'] = (df['operatingIncome']
                                     + df['depreciation'] + df['SaleOfPPE']).divide(df['TotalDebt'])

    df_ratios2['FreeOperCF_to_TotalDebt'] = (df2['operatingIncome'] + df2['depreciation']
                                             - df2['capitalExpenditures']).divide(df2['TotalDebt'])
    df_ratios2['FFO_to_TotalDebt'] = (df2['operatingIncome']
                                      + df2['depreciation'] + df2['SaleOfPPE']).divide(df2['TotalDebt'])

    tcc4_max = max(df_ratios2['FreeOperCF_to_TotalDebt'].max(),
                   df_ratios2['FFO_to_TotalDebt'].max())

    ratios_melt = pd.melt(df_ratios, id_vars=['ticker', 'date', 'TCC', 'name'])

    ratios_melt['date'] = pd.to_datetime(ratios_melt['date'])

    ratios_melt_cols4 = ratios_melt[ratios_melt.variable.isin(['FreeOperCF_to_TotalDebt', 'FFO_to_TotalDebt'])]

    color_discrete_map = my_palette2
    
    ratios_melt_cols4['Date_formated']=ratios_melt_cols4['date'].astype(str).str[0:4]

    if (Lambda < 0.8):
        fig7 = px.bar(ratios_melt_cols4, x="Date_formated", y="value", facet_row='ticker',
                          facet_col='TCC', hover_name='name',
                          labels={
                                  "value":"Ratio",
                                  "variable":" ",
                                  "Date_formated": ""
                             },
                          color="variable", range_y=[-0.1 * tcc4_max, 1.1 * tcc4_max],
                          color_discrete_map=color_discrete_map, barmode="group",text_auto='.2f')
    else:
        fig7 = px.bar(ratios_melt_cols4, x="Date_formated", y="value", facet_row='ticker',
                          facet_col='TCC', hover_name='name',
                          labels={
                                  "value":"Ratio",
                                  "variable":" ",
                                  "Date_formated": ""
                             },
                          color="variable", color_discrete_map=color_discrete_map, barmode="group",text_auto='.2f')

    fig7.update_layout(plot_style())
    fig7.update_layout(
        legend=dict(
            x=0,
            y=1,
            traceorder="reversed",
            bgcolor="#ffffff",
            bordercolor="Black",
            borderwidth=2,
            title=None,
        )
    )
                         
    fig7.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig7.update_annotations(font_size=25)

    return fig7

########################## To Do ###############################

def bar3_compare(ticks, Lambda):
    my_palette3 ={'Return_on_Capital': '#FFA41B', 'ebitda_to_Revenue': '#4C71DD'}
    import pandas as pd
    import plotly.express as px
    from plotly.offline import plot

    tickers_list = ticks

    df0 = concat(tickers_list[0])
    df0['ebitda'] = df0['operatingIncome'] + df0['depreciation']

    df1 = df0.copy()

    df2 = df0.copy()

    df0['TCC'] = 'Without TCC'

    df1['TCC'] = 'With TCC'

    df1['TotalDebt'] = df0['TotalDebt'].mul(1 - Lambda)
    df1['interestExpense'] = df0['interestExpense'].mul(1 - Lambda)
    df1['operatingIncome'] = df0['operatingIncome'] - abs(df0['interestExpense']).mul(Lambda)
    df1['ebitda'] = df0['ebitda'] - abs(df0['interestExpense']).mul(Lambda)

    df = pd.concat([df0, df1], axis=0)

    df['date'] = pd.to_datetime(df['date'])

    df_ratios = df[['ticker', 'date', 'TCC', 'name']]

    # df = df[['ticker', 'date', 'interestExpense', 'TotalDebt', 'ebitda', 'operatingIncome']]

    df_ratios['Return_on_Capital'] = (df['operatingIncome']
                                      - df['incomeTaxExpense']).divide(df['totalStockholderEquity']
                                                                       + df['TotalDebt'] - df['cash'])
    df_ratios['ebitda_to_Revenue'] = (df['operatingIncome'] + df['depreciation']).divide(df['totalRevenue'])

    ratios_melt = pd.melt(df_ratios, id_vars=['ticker', 'date', 'TCC', 'name'])

    ratios_melt['date'] = pd.to_datetime(ratios_melt['date'])

    ratios_melt_cols5 = ratios_melt[ratios_melt.variable.isin(['Return_on_Capital', 'ebitda_to_Revenue'])]

    color_discrete_map = my_palette3
    
    ratios_melt_cols5['Date_formated']=ratios_melt_cols5['date'].astype(str).str[0:4]

    fig = px.bar(ratios_melt_cols5, x="Date_formated", y="value", facet_row='ticker',
                      facet_col='TCC', hover_name='name',
                         labels={
                                          "value":"Ratio",
                                          "variable":" ",
                                          "Date_formated": ""
                                     },
                      color="variable", color_discrete_map=color_discrete_map, barmode="group",text_auto='.2f')

    fig.update_layout(plot_style())
    fig.update_layout(
        legend=dict(
            x=0,
            y=1,
            traceorder="reversed",
            bgcolor="#ffffff",
            bordercolor="Black",
            borderwidth=2,
            title=None,
        )
    )
                         
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_annotations(font_size=25)

    return fig

################# To Do #####################

def bar5_compare(ticks, Lambda):
    tickers_list=ticks  
    
    my_palette3 ={'Inventory ($ Millions)': '#FFA41B', 'Cash ($ Millions)': '#4C71DD'}
    
    df0 = concat(tickers_list[0])
    
    df1 = df0.copy()
    
    
    df0['TCC'] = 'Without TCC'
    
    df1['TCC'] = 'With TCC'
    
    df0['Inventory ($ Millions)']=df0['inventory'].mul(1/1000000)
    df1['Inventory ($ Millions)']=df0['inventory'].mul((1-Lambda)/1000000)
    
    df0['Cash ($ Millions)']=df0['cash'].mul(1/1000000)
    df1['Cash ($ Millions)']=(df0['cash'].mul((1/1000000)).add(df0['Inventory ($ Millions)'].mul((Lambda))))
    
    df = pd.concat([df0, df1], axis=0)
    
    df['date'] = pd.to_datetime(df['date'])
    
    df['Date_formated']=df['date'].astype(str).str[0:4]
    
    df_melt = pd.melt(df, id_vars=['ticker', 'Date_formated','name','TCC'])
    
    df_melt_cols = df_melt[df_melt.variable.isin(['Inventory ($ Millions)', 'Cash ($ Millions)'])]

    color_discrete_map=my_palette1
    
    tcc3_bar = px.bar(df_melt_cols, x="Date_formated", y="value", facet_row='ticker',
                      facet_col='TCC', hover_name='name',
                      labels={
                          "value":"Amount ($ Millions)",
                          "variable":" ",
                          "Date_formated": ""
                     },
                      color="variable", color_discrete_map= my_palette3,
                      barmode="group",text_auto='.2s')
    
    tcc3_bar.update_layout(plot_style())
    tcc3_bar.update_layout(
        legend=dict(
            x=0,
            y=1,
            traceorder="reversed",
            bgcolor="#ffffff",
            bordercolor="Black",
            borderwidth=2,
            title=None,
        )
    )
                         
    tcc3_bar.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    tcc3_bar.update_annotations(font_size=25)
    
    return tcc3_bar

########## To Do ##################

def update_ratios1(ticks):    
    
    ratios = ratios_(ticks[0])
    
    ratios['Date_formated']=ratios['date'].astype(str).str[0:4]
    
    tickers_list = ticks
    
    #ratios1 = ratios.drop(['date2'], axis=1)

    ratios_melt = pd.melt(ratios, id_vars=['ticker', 'Date_formated'])

    #ratios_melt['date2'] = pd.to_datetime(ratios_melt['date2'])

    ratios_melt_cols = ratios_melt[ratios_melt.variable.isin(['DIO', 'DPO', 'DSO'])]

    ratios_melt_cols_ticks = ratios_melt_cols[ratios_melt_cols.ticker.isin(tickers_list)].dropna()
        
    ratiosfig = px.bar(ratios_melt_cols_ticks, x="Date_formated", y="value",
                         color="variable", 
                        labels={
                                          "value":"Number of days",
                                          "variable":" ",
                                          "Date_formated": ""
                                     },
                         barmode="group", facet_row="ticker",text_auto='.0f',text="variable")
    ratiosfig.update_layout(plot_style())
    
    ratiosfig.update_xaxes(tickvals = list(ratios_melt_cols_ticks['Date_formated']))
    
    ratiosfig.update_layout(xaxis = dict(tickfont = dict(size=25)))
    
    ratiosfig.update_layout(font=dict(size=12))
    
    newnames = {'DIO': 'Days Inventory Outstanding',
                'DPO': 'Days Payable Outstanding',
                'DSO': 'Days Sales Outstanding'}
    ratiosfig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                          legendgroup = newnames[t.name],
                                          hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                         )
                      )

    ratiosfig.update_layout(legend=dict(
        orientation="h",
        title= None,
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        font=dict(
            size=13,
            color="black"
        ),
    ))
 
    return ratiosfig

# done by Alexandre 

def update_ratios2(ticks):    
    
    ratios = ratios_(ticks[0])
    
    ratios['Date_formated']=ratios['date'].astype(str).str[0:4]
    
    tickers_list = ticks
    
    #ratios1 = ratios.drop(['date2'], axis=1)

    ratios_melt = pd.melt(ratios, id_vars=['ticker', 'Date_formated'])

    #ratios_melt['date2'] = pd.to_datetime(ratios_melt['date2'])

    ratios_melt_cols = ratios_melt[ratios_melt.variable.isin(['InventoryTurnover','CCC'])]

    ratios_melt_cols_ticks = ratios_melt_cols[ratios_melt_cols.ticker.isin(tickers_list)].dropna()
        
    ratiosfig = px.bar(ratios_melt_cols_ticks, x="Date_formated", y="value",
                         color="variable", 
                        labels={
                                          "value":"Number of days",
                                          "variable":" ",
                                          "Date_formated": ""
                                     },
                         barmode="group", facet_row="ticker",text_auto='.0f',text="variable")
    ratiosfig.update_layout(plot_style())
    
    ratiosfig.update_xaxes(tickvals = list(ratios_melt_cols_ticks['Date_formated']))
    
    ratiosfig.update_layout(xaxis = dict(tickfont = dict(size=25)))
    
    ratiosfig.update_layout(font=dict(size=12))
    
    newnames = {'InventoryTurnover': 'Inventory Turnover',
                'CCC': 'Cash Convertion Cycle'}
    ratiosfig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                          legendgroup = newnames[t.name],
                                          hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                         )
                      )

    ratiosfig.update_layout(legend=dict(
        orientation="h",
        title= None,
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        font=dict(
            size=13,
            color="black"
        ),
    ))
 
    return ratiosfig

def round_didgit(dataset,variable,round_lvl):
    for index, row in dataset.iterrows():
        if math.isnan(dataset[variable].loc[index]):
            pass
        else:
            dataset[variable].loc[index]=round(dataset[variable].loc[index],round_lvl)

def av_indus(vari,tck,sctr):
    df_industry=df_annual_new[df_annual_new['sector']==sctr]
    tckl=[tck]
    df_tck=concat(tckl[0])
    tck_list=set(df_industry['ticker'])
    list_av=[]
    list_av_mod=[]
    list_tck_indus_av=[]
    list_date=[]

    for i in range(df_tck.shape[0]):
        list_av.append([])
        list_tck_indus_av.append('('+sctr+')'+' Average')
        list_date.append(df_tck['date'].iloc[i])

    for tick in tck_list:
        if df_industry[df_industry['ticker']==tick].shape[0] == df_tck.shape[0]:
            for i in range(df_tck.shape[0]):
                list_av[i].append(df_industry[df_industry['ticker']==tick][vari].iloc[i])
                
    for i in range(len(list_av)):
        new_list = [item for item in list_av[i] if not(math.isnan(item)) == True]
        list_av_mod.append(int((sum(new_list)/len(new_list))))
        
    for i in range(df_tck.shape[0]):
        list_av_mod.append(df_tck[vari].iloc[i])
        list_tck_indus_av.append(tck)
        list_date.append(df_tck['date'].iloc[i])    


    df_plot=pd.DataFrame({str(vari):list_av_mod, 'ticker':list_tck_indus_av, 'date':list_date})
    
    df_plot['Date_formated']=df_plot['date'].astype(str).str[0:4]
    
    df_plot[vari]=(df_plot[vari].mul(1/1000000))
    
    round_didgit(df_plot,vari,0)
    
    figa=px.line(df_plot, x="Date_formated", 
                 labels={
                                      "Date_formated": "",
                                     vari:vari+ " ($ Millions)"
                                 },
                 y=vari, title=vari +' evolution',color='ticker',text=vari)
    
    figa.update_layout(plot_style())
    figa.update_xaxes(tickvals = list(df_plot['Date_formated']))
    figa.update_traces(textposition='top center')
    figa.update_layout(xaxis = dict(tickfont = dict(size=25)))
    figa.update_layout(legend=dict(
        orientation="h",
        title= None,
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        font=dict(
            size=18,
            color="black"
        ),
    ))
    
    return figa



class PDF2(FPDF):
    
    def pagedegarde(self,name_comp):
        self.add_page()
        # Arial bold 15
        self.set_font('Arial', 'B', 35)
        # Calculate width of title and position
        w = self.get_string_width('TCC Analysis report') + 6
        self.set_x((210 - w) / 2)

        self.set_fill_color(256, 256, 256)

        # Title
        self.cell(w, 20, 'TCC Analysis report', 0, 0, 'C', 1)
        # Line break
        self.ln(20)
        
        self.set_font('Arial', '', 30)
        print(name_comp)
        w = self.get_string_width(name_comp) + 6
        self.set_x((210 - w) / 2)
        self.cell(w, 20, name_comp, 0, 0, 'C', 1)
        # Line break
        self.ln(20)
        
        logo='https://raw.githubusercontent.com/Bayan2019/TCC_Dashboard/main/pictures/Logo_Curve_BG-White.png'
        self.image(logo, (210-120)/2, 100, 120)
        self.ln(20)
        
        '''self.set_font('Arial', '', 16)
        w1 = self.get_string_width('For further information contact our finance analysis team') + 6
        self.set_x((210 - w1) / 2)
        self.cell(w1, 80, 'For further information contact our finance analysis team', 0, 0, 'C', 1)'''
    
    def header(self):
        # Arial bold 15
        self.set_font('Arial', 'B', 20)
        # Calculate width of title and position
        w = self.get_string_width(title) + 6
        self.set_x((210 - w) / 2)

        self.set_fill_color(250, 160, 70)

        # Title
        self.cell(w, 9, title, 0, 0, 'C', 1)
        # Line break
        self.ln(20)

    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # Arial italic 8
        self.set_font('Arial', 'I', 8)
        # Text color in gray
        self.set_text_color(128)
        # Page number
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def chapter_title(self, num, label):
        # Arial 12
        self.set_font('Arial', '', 14)
        # Background color
        self.set_fill_color(227, 227, 227)
        # Title
        self.cell(0, 6, 'Section %d : %s' % (num, label), 0, 1, 'L', 1)
        # Line break
        self.ln(10)

    def chapter_body_compare(self,involvment,num,tck):
        # Read text file
        txt='TCC impact :'
        # Times 12
        self.set_font('Times', '', 12)
        # Output justified text
        self.multi_cell(0, 5, txt)
        
        nm1=str(str(num)+"_bar1.png")
        bar5_compare([tck],involvment).write_image(nm1)
        self.image(nm1, (210-180)/2, 48, 180)
        
        nm2=str(str(num)+"_bar2.png")
        bar2_compare([tck],involvment).write_image(nm2)
        self.image(nm2, (210-180)/2, 160, 180)
        
        self.add_page()
        
        nm3=str(str(num)+"_bar3.png")
        bar3_compare([tck],involvment).write_image(nm3)
        self.image(nm3, (210-180)/2, 48, 180)
        
        nm4=str(str(num)+"_bar4.png")
        update_tcc_bar3([tck],involvment).write_image(nm4)
        self.image(nm4, (210-180)/2, 160, 180)
        
        # Line break
        self.ln()
        # Mention in italics
        self.set_font('', 'I')
        

    def chapter_body_industry(self,num,sctr,tck):
        
        '''txt='Company vs industry :'
        self.set_font('Arial', 'B', 20)
        self.multi_cell(0, 5, txt)'''
        
        txt1='Industry sector : '+str(sctr)
        self.set_font('Arial', '', 18)
        self.multi_cell(0, 5, txt1)
        
        nm1=str(str(num)+"_comp1.png")
        av_indus('cash',tck,sctr).write_image(nm1)
        self.image(nm1, (210-160)/2, 48, 160)
        
        nm2=str(str(num)+"_comp2.png")
        av_indus('inventory',tck,sctr).write_image(nm2)
        self.image(nm2, (210-160)/2, 152, 160)

        # Line break
        self.ln()
        # Mention in italics
        self.set_font('', 'I')
        
        
    def print_chapter_company(self, num, title, tck):
        self.add_page()
        self.chapter_title(num, title)
        
        nm2=str(str(num)+"_ratio2.png")
        update_ratios1([tck]).write_image(nm2)
        self.image(nm2, (210-180)/2, 45, 180)
        
        nm3=str(str(num)+"_ratio2.png")
        update_ratios2([tck]).write_image(nm3)
        self.image(nm3, (210-180)/2, 162, 180)
        
        '''self.set_font('Arial','B', 12)
        # Calculate width of title and position
        w = self.get_string_width('DIO: Days Inventory Outstanding ') + 6
        self.set_x((210 - w) / 2)
        self.cell(w, 20, 'DIO: Days Inventory Outstanding ', 0, 0, 'C', 1)
        
        w1 = self.get_string_width('DPO: Days Payable Outstanding ') + 6
        self.set_x((210 - w1) / 2)
        self.cell(w1, 20, 'DPO: Days Payable Outstanding ', 0, 0, 'C', 1)
        
        w2 = self.get_string_width('DSO: Days Sales Outstanding ') + 6
        self.set_x((210 - w2) / 2)
        self.cell(w2, 20, 'DSO: Days Sales Outstanding ', 0, 0, 'C', 1)'''
        
    def print_chapter_industry(self, num, title,sctr,tck):
        self.add_page()
        self.chapter_title(num, title)
        self.chapter_body_industry(num,sctr,tck)

    def print_chapter_compare(self, num, title,involvment,tck):
        self.add_page()
        self.chapter_title(num, title)
        self.chapter_body_compare(involvment,num,tck)


def generate_report(tck,inv):
    nc=str(df_tickers[df_tickers['ticker']==tck]['name'].iloc[0])
    sector=df_annual_new.loc[df_annual_new.index[df_annual_new['ticker']==tck].tolist()[0],'sector']
    pdf = PDF2()
    #pdf.set_title(title)
    pdf.set_author('Alexandre Courtois')
    


    ###-----------------------------

    pdf.pagedegarde(nc)

    ###-----------------------------

    pdf.print_chapter_company(1,'Company information', tck)
    pdf.print_chapter_industry(2, 'Company state compare to industry ',sector,tck)
    pdf.print_chapter_compare(3, 'Inventory financing impact ' + '30%',0.3,tck)
    pdf.print_chapter_compare(4, 'Inventory financing impact '+ '50%',0.5,tck)
    pdf.print_chapter_compare(5, 'Inventory financing impact '+ str(inv)+'% (custom)',inv/100,tck)
    #from pathlib import Path
    #downloads_path = str(Path.home() / "Downloads")
    #path=downloads_path+'TCC-Report.pdf'
    pdf.output('Auto-generated-report_TCC.pdf', 'F')

In [ ]:
ucr_csv_annual = 'https://raw.githubusercontent.com/Bayan2019/TCC_Dashboard/main/Files_of_Data/fs_annual_ws.csv'
df_annual = pd.read_csv(ucr_csv_annual)
ucr_csv_annual_new = 'https://raw.githubusercontent.com/Bayan2019/TCC_Dashboard/main/Files_of_Data/fs_annual_new_ws.csv'
df_annual_new = pd.read_csv(ucr_csv_annual_new)
src1 = 'https://raw.githubusercontent.com/Bayan2019/TCC_Dashboard/main/Files_of_Data/list_comp_fs_yahoo_ws.csv'
df_tickers = pd.read_csv(src1)
my_palette1 ={'ebit_to_interestExpense': '#FFA41B', 'ebitda_to_interestExpense': '#4C71DD'}
title = 'CaaS Report'


def create_report(ticker_input,inv_prct):
    print('Report generator : '+ticker_input)
    try:
        generate_report(ticker_input,inv_prct)
        print('\n\nReport Generated')
        return True
    except IndexError:
        print('Error : Invalid ticker')
        return False

In [ ]:
def create_report_un(ticker_input,inv_prct):
    print('Report generator : '+ticker_input)
    try:
        generate_report_un(ticker_input,inv_prct)
        print('\n\nReport Generated')
        return True
    except IndexError:
        print('Error : Invalid ticker')
        return False
    

    
def generate_report_un(tck,inv):
    nc=get_information(tck)['name']
    sector=get_information(tck)['sector']
    pdf = PDF2()
    #pdf.set_title(title)
    pdf.set_author('Alexandre Courtois')

    ###-----------------------------

    pdf.pagedegarde(nc)

    ###-----------------------------

    pdf.print_chapter_company(1,'Company information', tck)
    pdf.print_chapter_industry(2, 'Company state compare to industry ',sector,tck)
    pdf.print_chapter_compare(3, 'Inventory financing impact ' + '30%',0.3,tck)
    pdf.print_chapter_compare(4, 'Inventory financing impact '+ '50%',0.5,tck)
    pdf.print_chapter_compare(5, 'Inventory financing impact '+ str(inv)+'% (custom)',inv/100,tck)
    #from pathlib import Path
    #downloads_path = str(Path.home() / "Downloads")
    #path=downloads_path+'TCC-Report.pdf'
    pdf.output('Auto-generated-report_TCC.pdf', 'F')

In [ ]:
generate_report_un('AME',10)

AME


TypeError: ignored